### Assigned variables

In [0]:
bucket = "s3://end-to-end-banking-data-pipeline"
layer = "bronze"

### Imported Issential Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### Loaded data from s3 Bronze Layer

In [0]:
customers_df = spark.read.format("csv")\
    .option("inferSchema", True)\
    .option("header", True)\
    .load("s3://end-to-end-banking-data-pipeline/bronze/raw_customers/")

### Checking Schema 

In [0]:
customers_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- branch: string (nullable = true)
 |-- kyc_status: string (nullable = true)



### Applying Transformation 

**Droping** **Duplicates**

In [0]:
customers_df = customers_df.dropDuplicates()

**Filtering only the completed kyc Users**

In [0]:
customers_df = customers_df.filter(col("kyc_status") == "COMPLETED")

### Creting flag for KYC Status

In [0]:
customers_df = customers_df.withColumn("flag",when(col("kyc_status") == "COMPLETED","Y").otherwise("N"))

### Calculated Age

In [0]:
customers_df = customers_df.withColumn("Age",floor(months_between(current_date(),col("dob"))/12))

### Implemented Age Group

In [0]:
customers_df = customers_df.withColumn("age_group",when(col("age") < 18, "Youth")\
    .when((col("age") >= 18) & (col("age") < 60), "Adult")\
        .otherwise("Senior")
)

### Write data to Silver Layer

In [0]:
customers_df.write.format("csv")\
    .partitionBy("branch")\
    .option("header", True)\
    .mode("overwrite")\
    .save("s3://end-to-end-banking-data-pipeline/silver/cleaned_customers/")

In [0]:
print("successfully written")

successfully written
